Install the mysql-connector-python and pandas

In [ ]:
# This script is intended to be run in a Jupyter notebook or similar environment
# You can install the required packages using pip
# Uncomment the following line if you haven't installed the packages yet
# !pip install openpyxl pandas mysql-connector-python

In [ ]:
import mysql.connector as connector
import pandas as pd     
import numpy as np

In [ ]:
# Set up the connection to the MySQL database
try:
    connection = connector.connect(
        host="localhost",
        user="****",
        password="***",
        database="littlelemon"
    )
    cursor = connection.cursor()
except connector.Error as er:
    print("Error Code ", er.errno)
    print("Error Message ", er.msg)

In [ ]:
# Read the Excel file and load the Orders sheet into a DataFrame
df = pd.read_excel("Raw Data/LittleLemonDB.xlsx", sheet_name="Orders")

In [ ]:
# Drop the first column
df.drop(columns=["Row Number"], inplace=True)

# Rename the columns to remove leading spaces
df.columns = df.columns.str.strip()

# Drop duplicate rows based on the "Order ID" column
df = df.drop_duplicates(subset = ["Order ID"])

# Remove non-numeric characters from the Order ID column and Customer ID column then change the type to int
df['Order ID'] = df['Order ID'].str.replace('-','').astype(int)
df['Customer ID'] = df['Customer ID'].str.replace('-','').astype(int)

In [ ]:
df.head(5)

In [ ]:
# Check the shape of the DataFrame after dropping duplicates
df.shape

## Scope of Work
This notebook will prcocess data from an messy Excel file then store clean datasets into the database. It is noted that the database schema already created using MySQL Workbench, so we do not create new tables here.
We already have five tables as below and need to fill in the data for those tables:
- Customers
- Menu_Items
- Deliveries
- Orders
- Bookings

### 1. Prepare data for customers table

In [ ]:
# Extract the relevant columns for customers
customers = df[["Customer ID", "Customer Name", "City", "Country"]].drop_duplicates()

Create customer table and add data into table

In [ ]:

# Insert data into the customers table
for i, row in customers.iterrows():
    sql = "insert into customers (CustomerID, Name, City, Country) values (%s, %s, %s, %s)"
    cursor.execute(sql,tuple(row))

connection.commit()

### 2. Prepare data for menu_items table

In [ ]:
# Create a MenuItems data frame
menu_items = df[["Course Name", "Starter Name", "Desert Name", "Drink", "Sides"]].drop_duplicates()

In [ ]:
# Add a new column "MenuItemID" to the menu_items DataFrame
menu_items.insert(0, "MenuItemID", range(1, len(menu_items) + 1))
menu_items

In [190]:

# Insert data into the menu_items table
for i, row in menu_items.iterrows():
    sql = "insert into menu_items (Menu_Items_ID,CourseName, StarterName, DesertName, Drink, Sides) values (%s, %s, %s, %s, %s, %s)"
    cursor.execute(sql, tuple(row))

connection.commit()


In [191]:
# Merge df with menu_items to get MenuItemID to create foreign key relationship
df = df.merge(menu_items, how="left", on=["Course Name", "Starter Name", "Desert Name", "Drink", "Sides"])

In [193]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(df.head(10).iloc[:, -4:])

       Desert Name              Drink         Sides  MenuItemID
0    Greek yoghurt  Athens White wine         Tapas           1
1        Ice cream     Corfu Red Wine  Potato salad           2
2       Cheesecake     Italian Coffee    Bruschetta           3
3         Affogato      Roma Red wine      Focaccia           4
4  Turkish yoghurt  Ankara White Wine     Meatballs           5
5          Baklava     Turkish Coffee         Fries           6
6    Greek yoghurt  Athens White wine         Tapas           1
7        Ice cream     Corfu Red Wine  Potato salad           2
8       Cheesecake     Italian Coffee    Bruschetta           3
9         Affogato      Roma Red wine      Focaccia           4


### 3. Prepare data for Order table

In [194]:
# Create Orders data frame
Orders = df[["Order ID", "Order Date", "Customer ID", "Cost", "Sales", "Discount", "MenuItemID", "Quantity"]].drop_duplicates()

In [195]:
# Check data types of the Order Date column
print(type(Orders["Order Date"][0]))

# Convert the Order Date to a string in the format YYYY-MM-DD
Orders["Order Date"] = Orders["Order Date"].dt.strftime('%Y-%m-%d')

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [196]:

# Insert data into the orders table
for i, row in Orders.iterrows():
    sql = "insert into orders (OrderID, OrderDate, CustomerID, Cost, Sales, Discount, MenuItemID, Quantity) values (%s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.execute(sql, tuple(row))

In [197]:
connection.commit()

### 4. Prepare data for deliveries table

In [205]:
# Extract the relevant columns for Deliveries
deliveries = df[["Delivery Date", "Order ID"]].drop_duplicates()

In [206]:
# Convert the Order Date to a string in the format YYYY-MM-DD
deliveries["Delivery Date"] = deliveries["Delivery Date"].dt.strftime('%Y-%m-%d')

In [207]:
# Delivery status is not in the original data, so we will create a new column with 
# random value of "Not initiate", "Delivered", "In Progress", "Cancelled"
status = ["Not initiate", "Delivered", "In Progress", "Cancelled"]
deliveries["Delivery Status"] = np.random.choice(status, size=len(deliveries))
deliveries["Deliveries_ID"] = range(1, len(deliveries) + 1)

In [208]:
deliveries.head(5)

,Delivery Date,Order ID,Delivery Status,Deliveries_ID
0,2020-03-26,543666861,In Progress,1
1,2020-07-17,637613686,Not initiate,2
2,2020-04-24,653516434,Not initiate,3
3,2020-04-13,369172834,Cancelled,4
4,2021-02-02,861149232,Not initiate,5


In [209]:
# Insert data into the deliveries table
for i, row in deliveries.iterrows():
    sql = "insert into deliveries (DeliveryDate, OrderID, DeliveryStatus, Deliveries_ID) values (%s, %s, %s, %s)"
    cursor.execute(sql, tuple(row))


In [210]:
connection.commit()

### 5.Prepare data for Booking table

In [211]:
# Create Bookings data frame
bookings = df[["Customer ID"]][:20].drop_duplicates()

In [212]:
bookings['BookingID'] = range(1, len(bookings) + 1)
bookings['BookingDate'] = pd.to_datetime('2023-10-01') + pd.to_timedelta(np.random.randint(0, 30, size=len(bookings)), unit='d')
bookings['TableNo'] = np.random.randint(1, 11, size=len(bookings))

In [213]:
# Rearrange the columns to match the database schema
bookings = bookings.reindex(columns=["BookingID", "Customer ID", "BookingDate", "TableNo"])
bookings.head(5)

,BookingID,Customer ID,BookingDate,TableNo
0,1,720557985,2023-10-19,8
1,2,653530657,2023-10-09,4
2,3,908766799,2023-10-02,6
3,4,738734827,2023-10-04,2
4,5,809275246,2023-10-10,3


In [214]:
# Convert the Booking Date to a string in the format YYYY-MM-DD
bookings["BookingDate"] = bookings["BookingDate"].dt.strftime('%Y-%m-%d')

In [215]:
# Insert data into the bookings table
for i, row in bookings.iterrows():
    sql = "insert into bookings (BookingID, CustomerID, BookingDate, TableNo) values (%s, %s, %s, %s)"
    cursor.execute(sql, tuple(row))

In [216]:
connection.commit()

In [217]:
connection.close()

The database is ready for experiment and analysis.